# Week3 (Wuwei Zhang)

In [1]:
import numpy as np
# import matplotlib.pyplot as plt
from mnist import MNIST

def load_dataset():
    mndata = MNIST('C:/Users/Zhang/python-mnist/data')
    A_train, labels_train = map(np.array, mndata.load_training()) 
    A_test, labels_test = map(np.array, mndata.load_testing()) 
    A_train = A_train/255.0
    A_test = A_test/255.0
    
    return A_train, A_test, labels_train, labels_test

A_train, A_test, labels_train, labels_test = load_dataset()


B_labels_train = np.zeros((60000, 10))
for i in range(len(labels_train)):
    B_labels_train[i, labels_train[i]] = 1
B_labels_test = np.zeros((10000, 10))
for j in range(len(labels_test)):
    B_labels_test[j, labels_test[j]] = 1

## Use Ridge by built-in function

In [3]:
from sklearn.linear_model import Ridge
lam = [0.01, 0.02, 0.04, 0.08]
accuracy = np.zeros(4)

for i in range(len(lam)):
    clf = Ridge(alpha=lam[i])
    clf.fit(A_train, B_labels_train)
    X = clf.coef_.T
    B_test_predicted = np.matmul(A_test, clf.coef_.T)
    labels_test_predicted = np.argmax(B_test_predicted, axis = 1)
    accuracy[i] = 1-np.count_nonzero(labels_test - labels_test_predicted)/10000
print(accuracy)

[0.8117 0.8117 0.8117 0.8117]


## Solve X using ridge regression by hand
$\widehat{X} = (A^TA + \lambda I)^{-1} A^TB$

In [4]:
lam = [0.01, 0.02, 0.04, 0.08]
my_accuracy = np.zeros(4)

for i in range(len(lam)):
    inverse = np.linalg.pinv(A_train.T@A_train + lam[i]*np.identity(784))
    # Compute solution X by hand
    my_X = inverse@A_train.T@B_labels_train 
    B_test_predicted = np.matmul(A_test, my_X)
    labels_test_predicted = np.argmax(B_test_predicted, axis = 1)
    my_accuracy[i] = 1-np.count_nonzero(labels_test - labels_test_predicted)/10000
print(my_accuracy)

[0.8533 0.8533 0.8533 0.8534]


## my functions using ridge regression

In [5]:
def compute_X(A, B, lam):
    inverse = np.linalg.pinv(A.T@A + lam*np.identity(A.shape[1]))   
    my_X = inverse@A_train.T@B_labels_train
    return my_X
    
def compute_labels(A, X):    
    B_predicted = np.matmul(A, X)
    labels_predicted = np.argmax(B_predicted, axis = 1)
    return labels_predicted
    
    
X = compute_X(A_train, B_labels_train, 10^(-4))    
labels_train_predicted = compute_labels(A_train, X)
train_error = np.count_nonzero(labels_train - labels_train_predicted)/60000  
labels_test_predicted = compute_labels(A_test, X)
test_error = np.count_nonzero(labels_test - labels_test_predicted)/10000
print("train error: ", train_error)
print("test error: ", test_error)

train error:  0.18425
test error:  0.1793


* I don't know why I got 18% error when using lam = 10^(-4), but got 15% error when using lam = [0.01, 0.02, 0.04, 0.08]